# L1C Formatting Eploration
Stephen Whetzel

In [1]:
import pandas as pd
import h5py
from pyhdf.SD import SD, SDC
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
# os.chdir('C:\\Users\\whetz\\Documents\\UVA MSDS\\NASA\\hdf_files')
os.chdir('/Users/jackpeele/Documents/Data Science/NASA/HDF')
import icare

In [2]:
# os.chdir('C:\\Users\\whetz\\Documents\\UVA MSDS\\NASA\\hdf_files')
session = icare.ICARESession('/Users/jackpeele/Documents/Data Science/NASA/HDF')

ICARE Username: jpeele
ICARE Password: ···············


In [3]:
path='PARASOL/L1_B-HDF.v1.00/2008/2008_06_01'
file_list = list(session.listdir(path))
file_path = path + '/' + file_list[0]
session.get_file(file_path)

'/Users/jackpeele/Documents/Data Science/NASA/HDF/PARASOL/L1_B-HDF.v1.00/2008/2008_06_01/POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5'

In [4]:
f = h5py.File('PARASOL/L1_B-HDF.v1.00/2008/2008_06_01/POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5', 'r')

In [5]:
file_path

'PARASOL/L1_B-HDF.v1.00/2008/2008_06_01/POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5'

In [6]:
f

<HDF5 file "POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5" (mode r)>

In [7]:
cloud_arr = np.array(f['Data_Fields']['cloud_indicator'])
np.unique(cloud_arr[np.where(cloud_arr != 255)])

array([  0,  50, 100], dtype=uint8)

In [8]:
list(f['Data_Directional_Fields'].keys())

['CCD_column',
 'CCD_row',
 'I1020NP',
 'I443NP',
 'I490P',
 'I565NP',
 'I670P',
 'I763NP',
 'I765NP',
 'I865P',
 'I910NP',
 'Q490P',
 'Q670P',
 'Q865P',
 'U490P',
 'U670P',
 'U865P',
 'delta_thetav.cosphi',
 'delta_thetav.sinphi',
 'phi',
 'sequence_number',
 'thetas',
 'thetav']

In [9]:
list(f['Geolocation_Fields'])

['Latitude',
 'Longitude',
 'column_number',
 'land_sea_flag',
 'row_number',
 'surface_altitude']

In [10]:
list(f['Data_Fields'])

['Nviews', 'cloud_indicator', 'phis']

In [11]:
list(f['Quality_Flags_Directional_Fields'])

['Quality_Flags_01',
 'Quality_Flags_02',
 'Quality_Flags_03',
 'Quality_Flags_04',
 'Quality_Flags_05',
 'Quality_Flags_06',
 'Quality_Flags_07',
 'Quality_Flags_08',
 'Quality_Flags_09',
 'Quality_Flags_10',
 'Quality_Flags_11',
 'Quality_Flags_12',
 'Quality_Flags_13',
 'Quality_Flags_14']

In [12]:
temp = f['Quality_Flags_Directional_Fields']
np.array(temp['Quality_Flags_01']).shape

(3240, 6480, 16)

In [13]:
theta_scale = .0015
thetas_ds = f['Data_Directional_Fields']['thetas']
thetas = np.array(thetas_ds)
thetas[np.where(thetas == 65535)] = np.nan
thetas = thetas*theta_scale
thetas

ValueError: cannot convert float NaN to integer

In [14]:
angles = np.unique(np.round(thetas))
angles

array([    0,  7495,  7503, ..., 60494, 60515, 65535], dtype=uint16)

In [15]:
# phi_scale = .006
ds = f['Data_Directional_Fields']['I490P']
arr = np.array(ds)
# inds = np.where(arr != 0)
thetas

array([[[65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        ...,
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535]],

       [[65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        ...,
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535]],

       [[65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        ...,
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 655

In [16]:
thetas.shape

(3240, 6480, 16)

In [17]:
temp = thetas[np.where(thetas != 0)]
print(np.min(temp))
print(np.max(temp))

7495
65535


In [18]:
ds = f['Data_Directional_Fields']['phi']
ds.attrs['scale_factor']
list(ds.attrs)

['DIMENSION_LABELS',
 'HDF_Calibration_Equation',
 'Num_Fill',
 'Num_Valid',
 'Num_missing_value',
 '_FillValue',
 'actual_range',
 'add_offset',
 'add_offset_err',
 'long_name',
 'missing_value',
 'scale_factor',
 'scale_factor_err',
 'units']

# NETCDF4

In [16]:
os.chdir('C:\\Users\\whetz\\Documents\\UVA MSDS\\NASA\\hdf_files\\NCDF_Files')
from netCDF4 import Dataset
import netCDF4
nc = Dataset("PACE_SPEXONE_SIM.20170115T015622.L1C.5km.nc", "r", format="NETCDF4")
nc.groups.keys()

dict_keys(['bin_attributes', 'geolocation_data', 'observation_data', 'sensor_views_bands'])

In [17]:
nc.groups['sensor_views_bands'].variables.keys()

dict_keys(['intensity_bandpasses', 'intensity_F0', 'intensity_wavelengths', 'polarization_bandpasses', 'polarization_F0', 'polarization_wavelengths', 'view_angles'])

In [18]:
view_angles = np.array(nc.groups['sensor_views_bands']['view_angles'])
print(view_angles.shape)
print(view_angles)
nc.groups['sensor_views_bands']['view_angles']

(5,)
[58.06579  22.652498  4.424599 22.652498 58.06579 ]


<class 'netCDF4._netCDF4.Variable'>
float32 view_angles(number_of_views)
    long_name: Along-track view angles for sensor
    units: degrees
    _FillValue: -999.0
    valid_min: -89.0
    valid_max: 89.0
path = /sensor_views_bands
unlimited dimensions: 
current shape = (5,)
filling on

In [19]:
solar_zenith = nc.groups['geolocation_data']['solar_zenith']

In [20]:
wavelengths = np.array(nc.groups['sensor_views_bands']['intensity_wavelengths'])
print(wavelengths.shape)
wavelengths

(5, 400)


array([[380.    , 381.0125, 382.025 , ..., 781.9625, 782.975 , 783.9875],
       [380.    , 381.0125, 382.025 , ..., 781.9625, 782.975 , 783.9875],
       [380.    , 381.0125, 382.025 , ..., 781.9625, 782.975 , 783.9875],
       [380.    , 381.0125, 382.025 , ..., 781.9625, 782.975 , 783.9875],
       [380.    , 381.0125, 382.025 , ..., 781.9625, 782.975 , 783.9875]],
      dtype=float32)

In [27]:
obs_data = nc.groups['observation_data']

In [22]:
varname = 'DOLP_noise'
variable = obs_data[varname]
I_nf = np.array(variable)
print(I_nf.shape)
variable

(3568, 20, 5, 50)


<class 'netCDF4._netCDF4.Variable'>
float32 DOLP_noise(bins_along_track, bins_across_track, number_of_views, polarization_bands_per_view)
    long_name: Random noise of DOLP in bin
    _FillValue: -999.0
    valid_min: 0.0
    valid_max: 1.0
path = /observation_data
unlimited dimensions: 
current shape = (3568, 20, 5, 50)
filling on

In [23]:
I_nf[0][0][0]

array([0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003,
       0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003,
       0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003,
       0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003,
       0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003,
       0.003, 0.003, 0.003, 0.003, 0.003], dtype=float32)

In [24]:
lon = nc.groups['geolocation_data'].variables['longitude']
np.array(lon)

array([[-171.90573, -171.78265, -171.65993, ..., -169.85051, -169.73131,
        -169.6122 ],
       [-171.95668, -171.8338 , -171.71126, ..., -169.9044 , -169.78535,
        -169.6664 ],
       [-172.00748, -171.88477, -171.7624 , ..., -169.9581 , -169.83922,
        -169.72043],
       ...,
       [ 131.7372 ,  131.85594,  131.97478, ...,  133.77827,  133.90059,
         134.02324],
       [ 131.68323,  131.80214,  131.92113, ...,  133.72719,  133.84967,
         133.9725 ],
       [ 131.62909,  131.74815,  131.86731, ...,  133.67592,  133.79858,
         133.9216 ]], dtype=float32)

# Create Example File (nc)

In [252]:
# Get variables from existing .nc file
lons = nc.groups['geolocation_data']['longitude']
lats = nc.groups['geolocation_data']['latitude']
rel_azimuth = nc.groups['geolocation_data']['solar_azimuth']

In [287]:
# Dictionary with all of our categories and variables
array_dict = {
    'geolocation_data':[lons, lats, rel_azimuth]
}

# dictionary to be populated with all of our variable data/metadata
variable_dict = {}

# Loop through categories
for cat in array_dict.keys():
    
    # Create category level in variable dict
    variable_dict[cat] = {}
    
    # Loop through variables and populate the dictionary
    for var in array_dict[cat]:
        name = var.name
        
        # Create variable dictionary 
        variable_dict[cat][name] = {}
        
        # Get scaling factor
        try:
            scale = var.scale_factor
        except:
            scale = 1
        
        # Populate variable dictionary with data & metadata
        variable_dict[cat][name]['scale'] = scale
        variable_dict[cat][name]['long_name'] = var.long_name
        variable_dict[cat][name]['units'] = var.units
        variable_dict[cat][name]['fill_value'] = var._FillValue
        variable_dict[cat][name]['shape'] = var.shape
        variable_dict[cat][name]['array'] = np.array(var)
        
variable_dict

{'geolocation_data': {'longitude': {'scale': 1,
   'long_name': 'Longitudes of bin locations',
   'units': 'degrees_east',
   'fill_value': -999.0,
   'shape': (3568, 20),
   'array': array([[-171.90573, -171.78265, -171.65993, ..., -169.85051, -169.73131,
           -169.6122 ],
          [-171.95668, -171.8338 , -171.71126, ..., -169.9044 , -169.78535,
           -169.6664 ],
          [-172.00748, -171.88477, -171.7624 , ..., -169.9581 , -169.83922,
           -169.72043],
          ...,
          [ 131.7372 ,  131.85594,  131.97478, ...,  133.77827,  133.90059,
            134.02324],
          [ 131.68323,  131.80214,  131.92113, ...,  133.72719,  133.84967,
            133.9725 ],
          [ 131.62909,  131.74815,  131.86731, ...,  133.67592,  133.79858,
            133.9216 ]], dtype=float32)},
  'latitude': {'scale': 1,
   'long_name': 'Latitudes of bin locations',
   'units': 'degrees_north',
   'fill_value': -999.0,
   'shape': (3568, 20),
   'array': array([[-70.505066, -70

In [296]:
example.close()
example = Dataset('example.nc', mode='w')

for cat in variable_dict.keys():

    # Create the category group to store the variables
    example.createGroup(cat)
    
    for var in variable_dict[cat].keys():
        print(var)
    
        # Fill the dimension with variables
        dimensions = []
        for i in range(len(variable_dict[cat][var]['array'].shape)):
            dim_name = f'{var}_{i}'
            example.createDimension(dim_name, size=None)
            dimensions.append(dim_name)
        
        # Create the variable instance
        example[cat].createVariable(var, datatype='f4', dimensions=dimensions, fill_value=variable_dict[cat][var]['fill_value'])

        # Create variable metadata 
        example[cat][var].long_name = variable_dict[cat][var]['long_name']
        example[cat][var].units = variable_dict[cat][var]['units']
        example[cat][var].scale_factor = variable_dict[cat][var]['scale']
        
        # Create variable array 
        example[cat][var][:] = variable_dict[cat][var]['array']

longitude
latitude
solar_azimuth


In [301]:
np.array(example['geolocation_data']['longitude'])

array([[-171.90573, -171.78265, -171.65993, ..., -169.85051, -169.73131,
        -169.6122 ],
       [-171.95668, -171.8338 , -171.71126, ..., -169.9044 , -169.78535,
        -169.6664 ],
       [-172.00748, -171.88477, -171.7624 , ..., -169.9581 , -169.83922,
        -169.72043],
       ...,
       [ 131.7372 ,  131.85594,  131.97478, ...,  133.77827,  133.90059,
         134.02324],
       [ 131.68323,  131.80214,  131.92113, ...,  133.72719,  133.84967,
         133.9725 ],
       [ 131.62909,  131.74815,  131.86731, ...,  133.67592,  133.79858,
         133.9216 ]], dtype=float32)

In [293]:
example.dimensions

{'longitude_0': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'longitude_0', size = 0,
 'longitude_1': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'longitude_1', size = 0,
 'latitude_0': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'latitude_0', size = 0,
 'latitude_1': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'latitude_1', size = 0,
 'solar_azimuth_0': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'solar_azimuth_0', size = 0,
 'solar_azimuth_1': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'solar_azimuth_1', size = 0,
 'solar_azimuth_2': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'solar_azimuth_2', size = 0}

# Create example file from h5

In [19]:
f.keys()

<KeysViewHDF5 ['Data_Directional_Fields', 'Data_Fields', 'Geolocation_Fields', 'Quality_Flags_Directional_Fields']>

In [21]:
f['Data_Directional_Fields'].keys()

<KeysViewHDF5 ['CCD_column', 'CCD_row', 'I1020NP', 'I443NP', 'I490P', 'I565NP', 'I670P', 'I763NP', 'I765NP', 'I865P', 'I910NP', 'Q490P', 'Q670P', 'Q865P', 'U490P', 'U670P', 'U865P', 'delta_thetav.cosphi', 'delta_thetav.sinphi', 'phi', 'sequence_number', 'thetas', 'thetav']>

In [22]:
f['Geolocation_Fields'].keys()

<KeysViewHDF5 ['Latitude', 'Longitude', 'column_number', 'land_sea_flag', 'row_number', 'surface_altitude']>

In [33]:
f['Data_Fields'].keys()

<KeysViewHDF5 ['Nviews', 'cloud_indicator', 'phis']>

In [23]:
f['Geolocation_Fields']['Latitude'].shape

(3240, 6480)

In [32]:
print(f['Geolocation_Fields']['Longitude'].shape)
print(f['Geolocation_Fields']['Longitude'].shape)

(3240, 6480)
(3240, 6480)


In [35]:
print(f['Data_Directional_Fields']['I1020NP'].shape)
print(f['Data_Directional_Fields']['I443NP'].shape)

(3240, 6480, 16)
(3240, 6480, 16)


In [36]:
print(f['Data_Directional_Fields']['thetav'].shape)

(3240, 6480, 16)


In [38]:
print(f['Data_Directional_Fields']['I490P'].shape)

(3240, 6480, 16)


In [39]:
print(f['Data_Directional_Fields']['Q490P'].shape)

(3240, 6480, 16)


In [40]:
f['Data_Directional_Fields']['I490P']

<HDF5 dataset "I490P": shape (3240, 6480, 16), type "<i2">

In [42]:
i_fields = [field for field in list(f['Data_Directional_Fields'].keys()) if 'I' in field]
i_fields

['I1020NP',
 'I443NP',
 'I490P',
 'I565NP',
 'I670P',
 'I763NP',
 'I765NP',
 'I865P',
 'I910NP']

In [43]:
data_dir = f['Data_Directional_Fields']

In [46]:
temp = np.array(data_dir[i_fields[0]])
temp1 = np.array(data_dir[i_fields[1]])
temp2 = np.array()

In [62]:
np.stack((temp, temp1), axis=3).shape

(3240, 6480, 16, 2)

In [64]:
q_fields = [field for field in list(f['Data_Directional_Fields'].keys()) if 'Q' in field]
q_fields

['Q490P', 'Q670P', 'Q865P']

In [65]:
f['Data_Directional_Fields'].keys()

<KeysViewHDF5 ['CCD_column', 'CCD_row', 'I1020NP', 'I443NP', 'I490P', 'I565NP', 'I670P', 'I763NP', 'I765NP', 'I865P', 'I910NP', 'Q490P', 'Q670P', 'Q865P', 'U490P', 'U670P', 'U865P', 'delta_thetav.cosphi', 'delta_thetav.sinphi', 'phi', 'sequence_number', 'thetas', 'thetav']>

In [68]:
thetas = np.array(data_dir['thetav'])

In [69]:
thetas.shape

(3240, 6480, 16)

In [107]:
np.where(thetas != 65535)


(array([  25,   25,   25, ..., 2743, 2743, 2743]),
 array([3288, 3288, 3288, ..., 1930, 1930, 1930]),
 array([ 0,  1,  2, ..., 13, 14, 15]))

In [95]:
pd.DataFrame(np.where(thetas != 65535)[1]).head(70)

,0
0,3288
1,3288
2,3288
3,3288
4,3288
...,...
65,3285
66,3285
67,3285
68,3285


In [98]:
thetas[25][3288][0]

45556

In [96]:
thetas[26][3285]

array([45127,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=uint16)